In [ ]:
import pandas as pd
import time
import datetime
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
from datetime import timedelta

In [4]:
my_api_key = ''

In [5]:
df = pd.read_csv('C:/Users/rsh15/Desktop/seunghuni/lol_analysis/data/lol_api/high_tier_user_1009.csv')

In [5]:
df.head(5)

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,...,matches,win_rt,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,status
0,d1ff3466-6c02-33b0-8085-9fa6a8b82dd5,RANKED_SOLO_5x5,CHALLENGER,I,9c4m-A9pR8OcDZObVPWER3bPxfT-nZNUIDTpSzBGeiQ6OQ,쪼렙이다말로하자,1644,900,798,True,...,1698,0.530035,9c4m-A9pR8OcDZObVPWER3bPxfT-nZNUIDTpSzBGeiQ6OQ,A36uiPejeTR2gC_gSukPwWzlLZ2sUmvniqGN2bpFpB9b,alv2DWzNFI1ENQTHSqJ6qq0lNKYQfiJwxaJ3uxna8S35g0...,쪼렙이다말로하자,588.0,1.633427e+12,513.0,NaN
1,d1ff3466-6c02-33b0-8085-9fa6a8b82dd5,RANKED_SOLO_5x5,CHALLENGER,I,l3rb1MFE1niulYiKv3_iVRs-DLtWxYx78AL3kPQUH5In1ME,BRO Hoya,1432,411,329,True,...,740,0.555405,l3rb1MFE1niulYiKv3_iVRs-DLtWxYx78AL3kPQUH5In1ME,a88z5hluQZyUYSOAAmRMsiVUVdesMSTDlnpQybrhx7kHLeo,gxgvEsQApWB3HIiahee3WGYgY25ukt7gugfhy3zgmTOhrm...,BRO Hoya,4295.0,1.633491e+12,199.0,NaN
2,d1ff3466-6c02-33b0-8085-9fa6a8b82dd5,RANKED_SOLO_5x5,CHALLENGER,I,APY4jcjbzJDEKXSXpTdzijO8PB1iM9IKifid_GCmsXS49z...,못 찾을 외딴곳에,1401,754,656,True,...,1410,0.534752,APY4jcjbzJDEKXSXpTdzijO8PB1iM9IKifid_GCmsXS49z...,qQ4KSnKGOUAOx2REZJ4eMiqm09xsM3ASBFIZp4603w8A3i...,nKJmyy1JJmcFWIJFOMOxG4WcT8roU09OlPChgDjDV7yf0X...,못 찾을 외딴곳에,25.0,1.633420e+12,138.0,NaN
3,d1ff3466-6c02-33b0-8085-9fa6a8b82dd5,RANKED_SOLO_5x5,CHALLENGER,I,lgywzY3QbLlUyjguGLNOj8ixjmEFfAL2Z_lZmEz9l-3MZnE,DRX Kingen00,1393,691,596,True,...,1287,0.536908,lgywzY3QbLlUyjguGLNOj8ixjmEFfAL2Z_lZmEz9l-3MZnE,TWdo0TEm-7RoUVq8-GXYJseSLoV_GO8qFfCLpm9CT7XuTJQ,3E9rnBq_iUB11j77MJSSR26fRfTrI5xKVZN146ND0mftq9...,DRX Kingen00,4556.0,1.633588e+12,600.0,NaN
4,d1ff3466-6c02-33b0-8085-9fa6a8b82dd5,RANKED_SOLO_5x5,CHALLENGER,I,DvrWMf5MTWs83jo5q_6yFGDmw0jHkefxcFSmvNmNY1z-6DI,Let me sup,1342,954,865,True,...,1819,0.524464,DvrWMf5MTWs83jo5q_6yFGDmw0jHkefxcFSmvNmNY1z-6DI,JiD85qr3eQlGi62yssFEJI3WyUh_BDYqJ0wAKezMnLF474g,ioMVdITHQ9lBGKhqqnRQl_YparfQIedp4lyBvDtMy0XqN-...,Let me sup,5063.0,1.633290e+12,246.0,NaN


In [7]:
df = df[df['tier'].isin(['CHALLENGER'])]

In [8]:
df = df.dropna(subset=['id'],axis=0)
df = df.reset_index(drop=True)

In [9]:
df.isna().sum()

leagueId           0
queueType          0
tier               0
rank               0
summonerId         0
summonerName       0
leaguePoints       0
wins               0
losses             0
veteran            0
inactive           0
freshBlood         0
hotStreak          0
matches            0
win_rt             0
id                 0
accountId          0
puuid              0
name               0
profileIconId      0
revisionDate       0
summonerLevel      0
status           297
dtype: int64

### prepare

#### making timestamp

In [10]:
start_d = pd.date_range(start='2021/06/16', end='2021/10/10',freq='7D')
enddd_d = start_d[1:] - timedelta(days=1)
enddd_d = enddd_d.append(pd.DatetimeIndex(['2021-10-10']))

In [11]:
start_ts = []
enddd_ts = []
for st_d, en_d in zip(start_d,enddd_d):
    start_ts.append(int(time.mktime(datetime.datetime(
        st_d.year, st_d.month, st_d.day, 0, 0, 0).timetuple())))
    
    enddd_ts.append(int(time.mktime(datetime.datetime(
        en_d.year, en_d.month, en_d.day, 23, 59, 59).timetuple())))

#### making urls_list

In [ ]:
# 기본 url. [puid,startdate,enddate,api_key]

In [12]:
match_id_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?startTime={}&endTime={}&type=ranked&start=0&count=100&api_key={}'

In [14]:
urls_list = []
for summoner_puuid in df['puuid']:
    for starts,ends in zip(start_ts,enddd_ts):
        urls_list.append(match_id_url.format(summoner_puuid,starts,ends,my_api_key))

In [15]:
results = []
for url_now in tqdm(urls_list):
    
    req_text = requests.get(url_now).text
    
    if req_text == '{"status":{"message":"Rate limit exceeded","status_code":429}}':
        time.sleep(120)
        
        req_text = requests.get(url_now).text
        results.append(req_text)
    
    else:
        results.append(req_text)

100%|████████████████████████████████████████████████████████████████████████████| 5049/5049 [2:02:11<00:00,  1.45s/it]


In [16]:
[x for x in results if x == '{"status":{"message":"Rate limit exceeded","status_code":429}}']

[]

In [30]:
results_eval = pd.Series(results).apply(lambda x : eval(x))
results_eval = np.sum(results_eval)
results_eval[0:10]

['KR_5278595528',
 'KR_5278475396',
 'KR_5278358463',
 'KR_5278341320',
 'KR_5278291118',
 'KR_5278127451',
 'KR_5278089561',
 'KR_5277939404',
 'KR_5277974179',
 'KR_5277846999']

In [52]:
#results_df = pd.DataFrame(results_eval)
#results_df.to_csv('C:/Users/rsh15/Desktop/seunghuni/lol_analysis/data/lol_api/match_ids.csv',
                   # index=False,encoding='utf-8-sig')

In [11]:
results_df = pd.read_csv('C:/Users/rsh15/Desktop/seunghuni/lol_analysis/data/lol_api/match_ids.csv')
results_eval = results_df.iloc[:,0]
results_eval

0         KR_5278595528
1         KR_5278475396
2         KR_5278358463
3         KR_5278341320
4         KR_5278291118
              ...      
123462    KR_5494739323
123463    KR_5494785504
123464    KR_5494811475
123465    KR_5494770109
123466    KR_5494656051
Name: 0, Length: 123467, dtype: object

In [13]:
match_hist_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'

In [21]:
options = webdriver.ChromeOptions()
options.add_argument("headless")
options.add_argument("window-size=1920x1080")
options.add_argument("disable-gpu")
options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36")
options.add_argument("lang=ko_KR")

driver = webdriver.Chrome('C:/Users/rsh15/Desktop/seunghuni/selenium/chromedriver.exe',
                          options = options)

In [29]:
match_hist = []
i = 0
for match_cd in tqdm(np.unique(results_eval)):
    
    urll = match_hist_url.format(match_cd,my_api_key)

    if i == 99:
        time.sleep(120)
        
        driver.get(urll)
        match_hist.append(BeautifulSoup(driver.page_source).text)
    
        i = 0
        
    else:
        driver.get(urll)
        match_hist.append(BeautifulSoup(driver.page_source).text)
        
        i += 1

  7%|████▉                                                                   | 3699/54391 [1:29:00<20:19:51,  1.44s/it]


WebDriverException: Message: unknown error: net::ERR_CONNECTION_FAILED
  (Session info: headless chrome=94.0.4606.71)
